## Initial hashtag analysis
Tayae Rogers, 3/12/2024

In [1]:
import os
import csv
from collections import defaultdict
import pandas as pd
import re
import json

In [2]:
def list_of_hashtags(csv_in):
    """
    Takes hashtag CSV (our shared Google Sheets file) and returns a list of news accounts
    """
    file = f"{csv_in}.csv"
    hashtags = []
    with open(file, mode ='r')as file:
        csvFile = csv.reader(file)
        for lines in csvFile:
            hashtags.append(lines[0])
    return hashtags

In [3]:
wd = "/Users/tayaerogers/Documents/MEDSL/GitHub/CS315-Project-2" # put personal wd ending in "/CS315-Project-2" here
hashtag_wd = wd + "/analysis/hashtag_initial/"
hashtag_list = list_of_hashtags(os.path.join(hashtag_wd, 'hashtags_03112024'))

In [4]:
# path = os.path.dirname(os.path.abspath('hashtag_analysis_initial.ipynb'))
# path = path.rsplit('/',2)[0]
# path
curr_path = f"{wd}/pre-processing/metadata-csv"
curr_path

'/Users/tayaerogers/Documents/MEDSL/GitHub/CS315-Project-2/pre-processing/metadata-csv'

In [5]:
def get_hashtags(phrase):
    """"Takes a string phase and returns a list of hashtags
    """
    if isinstance(phrase,str):
        return re.findall(r'#(\S+)', phrase.lower())
    else:
        return []

In [6]:
def news_post_abundance(path):
    all_news = pd.DataFrame(columns = ["video_id", "author_username", "video_description", "hashtags", "suggested_words", "file_name"])
    stats_df = pd.DataFrame(columns = ["file_name", "full_length", "news_length", "prop_news"])
    prefixed = [filename for filename in os.listdir(path) if filename.startswith("Sec2Gr3")]

    for file in prefixed:
        meta_csv = pd.read_csv(os.path.join(path,file))
        meta_csv["hashtags"] = meta_csv["video_description"].apply(get_hashtags)

        meta_sub_df = meta_csv[["video_id", "author_username", "video_description", "hashtags", "suggested_words", "video_timestamp"]].copy()
        meta_sub_df["file_name"] = str(file)
        news_df = meta_sub_df[meta_sub_df["hashtags"].apply(lambda htlst: (any(hashtag in htlst for hashtag in hashtag_list) if isinstance(htlst, list) else False))]
        news_df['video_date'] = news_df['video_timestamp'].str[:10]

        allLen = meta_csv.shape[0]
        newsLen = news_df.shape[0]
        propNews = newsLen/allLen
        stats_df.loc[len(stats_df.index)] = [str(file), allLen, newsLen, propNews]  

        all_news = pd.concat([all_news, news_df])
    
    return (stats_df, all_news)

In [38]:
## Not working for some reason, but path stuff above is working
""" path = os.path.dirname(os.path.abspath('hashtag_analysis_initial.ipynb'))
path = path.rsplit('/',2)[0]
curr_path = f"{path}/pre-processing/metadata-csv"
curr_path """

' path = os.path.dirname(os.path.abspath(\'hashtag_analysis_initial.ipynb\'))\npath = path.rsplit(\'/\',2)[0]\ncurr_path = f"{path}/pre-processing/metadata-csv"\ncurr_path '

In [7]:
news_by_hashtag = news_post_abundance(curr_path)[1]
more_hashtags = news_by_hashtag["hashtags"].explode().to_list()

/var/folders/79/rqxxrf3n6zq58_z0ng_2hbhm0000gn/T/ipykernel_68585/3469459771.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['video_date'] = news_df['video_timestamp'].str[:10]
/var/folders/79/rqxxrf3n6zq58_z0ng_2hbhm0000gn/T/ipykernel_68585/3469459771.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['video_date'] = news_df['video_timestamp'].str[:10]
/var/folders/79/rqxxrf3n6zq58_z0ng_2hbhm0000gn/T/ipykernel_68585/3469459771.py:13: SettingWithCopyWarning: 
A value is trying to be se

In [8]:
news_by_hashtag

,video_id,author_username,video_description,hashtags,suggested_words,file_name,video_timestamp,video_date
134,7294807570915626283,lincolnsmamaaa_,THIS ONE IS GONNA BE HARD TO BEAT!!! 12 will d...,"[advent, adventcalendar, adventcalendar2023, a...",NaN,Sec2Gr3_77777.csv,2023-10-27T20:59:10,2023-10-27
341,6988185257426128133,gma,Morgan Wallen is speaking out to address using...,"[news, morganwallen]","morgan wallen racist video, morgan wallen, mor...",Sec2Gr3_77777.csv,2021-07-23T14:07:54,2021-07-23
53,7231173879845113131,getadjustednow,You should be doing THIS if you want to “grow ...,"[nycchiro, learnontiktok, getadjustednow, grow...","spine decompressing, how to get taller, stretc...",Sec2Gr3_77217.csv,2023-05-09T09:28:02,2023-05-09
143,7297290457330158894,sammyobeidthem,Add “ceasefire” to your wishlist immediately. ...,"[standupcomedy, geography, history, historytok]",NaN,Sec2Gr3_77217.csv,2023-11-03T13:34:17,2023-11-03
168,7303271447374466346,icedvanillatriplelatte,Im so tired of this argument #freepalestine #...,"[freepalestine, palestine, palestineisrael, fr...","gaza to the sea, zionists explained",Sec2Gr3_77217.csv,2023-11-19T15:23:29,2023-11-19
...,...,...,...,...,...,...,...,...
8829,7282735903003381010,jasminedinis2,In a world full of women teaching their childr...,"[homesteading, tradwife, motherhood, foryoupag...","jasmine dinis, Tradwife, women empowerment, Di...",Sec2Gr3_74721.csv,2023-09-25T08:14:50,2023-09-25
8864,7287317896370441514,ashly.antony,Ur fav couple #temple #templeuniversity #tailg...,"[temple, templeuniversity, tailgate, cherryand...",NaN,Sec2Gr3_74721.csv,2023-10-07T16:35:33,2023-10-07
8872,7287284369268411691,definitelynotforkpoker,Fuck it we ball #swag #michigan,"[swag, michigan]","bad roommates in college, Roommate TikTok, bad...",Sec2Gr3_74721.csv,2023-10-07T14:25:21,2023-10-07
8891,7285777286316510470,ctvnews,If you’re stuck in traffic on Hwy. 400 Tuesday...,"[breakingnews, truck, highway, ontario, gta, t...",NaN,Sec2Gr3_74721.csv,2023-10-03T12:56:59,2023-10-03


In [9]:
with open(f"{wd}/analysis/hashtag_initial/more_hashtags.json", 'w') as outf:
    json.dump(more_hashtags, outf)

In [10]:
news_by_hashtag.to_csv(f"{wd}/analysis/hashtag_initial/news_by_hashtag.csv", index=False)  